# Метод k-ближайших соседей в задаче классификации

** Метрика ** - функция пары объектов, удовлетворяющая для любых $x, y, z$ следующим соотношениям:

1. $d(x, y) \geq 0$

2. $d(x, y) = 0 \leftrightarrow x = y $

3. $d(x, y) = d(y, x)$

4. $d(x, y) \leq d(x, z) + d(x, y)$

** Примеры ** (кто знает, кто вспомнит?)

Пусть на признаковом пространстве задана метрика.

**Алгоритм:**

1. Получаем на вход объект
2. Находим в обучающей выборки k ближайших ко входному объектов
3. Относим входной объект к классу, доминирующему в выборке

Такой алгоритм будет корректно решать задачу классификации, если выполнена **гипотеза компактности**: рядом в произвольным объектом чаще встречаются объекты его класса.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn; 
from sklearn.linear_model import LinearRegression
from scipy import stats
import pylab as pl

seaborn.set()

## Ирисы

Начнем знакомство с алгоритмом на примере классического набора данных.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

n_samples, n_features = iris.data.shape

print('We have {} samples, {} features'.format(n_samples, n_features))
print('Target names are: {}, {}, {}'.format(*iris.target_names))
print('We are provided with the following features: {}, {}, {}, {}'.format(*iris.feature_names))

## Визуализация

Изобразим классы в зависимости от ширины наружной доли околоцветника и длины внутренней доли около цветника.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 'sepal width (cm)'
x_index = 1
# 'petal length (cm)'
y_index = 2

# this formatter will label the colorbar with the correct target names
formatter = plt.FuncFormatter(lambda i, *args: iris.target_names[int(i)])

plt.scatter(iris.data[:, x_index], iris.data[:, y_index],
            c=iris.target, cmap=plt.cm.get_cmap('RdYlBu', 3))
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.clim(-0.5, 2.5)
plt.xlabel(iris.feature_names[x_index])
plt.ylabel(iris.feature_names[y_index]);

Признаков немного, поэтому можно также перебрать все их пары.

In [ ]:
fig, axes = plt.subplots(4, 4)
fig.set_size_inches(20, 20)
formatter = plt.FuncFormatter(lambda i, *args: iris.target_names[int(i)])

for x_index in range(4):
    for y_index in range(4):
        if x_index != y_index:
            axes[x_index, y_index].scatter(iris.data[:, x_index], iris.data[:, y_index],
                                           c=iris.target, cmap=plt.cm.get_cmap('RdYlBu', 3))
            #axes[x_index, y_index].colorbar(ticks=[0, 1, 2], format=formatter)
            #plt.clim(-0.5, 2.5)
            axes[x_index, y_index].set_xlabel(iris.feature_names[x_index])
            axes[x_index, y_index].set_ylabel(iris.feature_names[y_index])
            
fig.subplots_adjust(right=0.8)

# Ближайшие соседи в действии

Помимо выбора числа соседей, параметры класса позволяют 
- определять важность объектов обучающей выборки (*weights*)
- определять алгоритмы поиска соседей (*algorithm*)
- определять метрику (*metric, p, metric_params*)

In [ ]:
from sklearn import neighbors, datasets

help(neighbors.KNeighborsClassifier)

Обучаемся

In [ ]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

# create the model
knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')

# fit the model
knn.fit(X, y)

Предсказываем

In [ ]:
# What kind of iris has 3cm x 5cm sepal and 4cm x 2cm petal?
X_pred = [3, 5, 4, 2]
result = knn.predict([X_pred, ])

print(iris.target_names[result])
print(iris.target_names)
print(knn.predict_proba([X_pred, ]))

*Упражнение.* Как выбрать число соседей? Подберите его с помощью кроссвалидации по схеме *Leave One Out*

## Анализ работы алгоритма

Визуализируем решающее правило

In [ ]:
from matplotlib.colors import ListedColormap

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

def plot_iris_knn():
    iris = datasets.load_iris()
    X = iris.data[:, :2]  # we only take the first two features. We could
                        # avoid this ugly slicing by using a two-dim dataset
    y = iris.target

    knn = neighbors.KNeighborsClassifier(n_neighbors=5)
    knn.fit(X, y)

    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    pl.figure()
    pl.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    pl.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    pl.xlabel('sepal length (cm)')
    pl.ylabel('sepal width (cm)')
    pl.axis('tight')
    
plot_iris_knn()

О чем стоит тут задуматься?

1. Сложность решающего правила
2. Допустимые размеры данных
3. Выбросы
4. Метрика

*Упражнение.* Нарисовать результат работы алгоритма, использующего другую метрику. Заметна ли разница?

In [ ]:
#Попробуйте другие метрики

## Проклятие размерности

Еще один важный момент, о котором не заставляют задуматься ирисы Фишера.

...

Один признак:

In [ ]:
from sklearn.cross_validation import cross_val_score

X_a = np.random.randn(100, 1)
X_a[:, 0] += -1
X_b = np.random.randn(100, 1)
X_b[:, 0] += 1

X = np.concatenate((X_a, X_b), axis = 0)

y = np.zeros(200)
y[:100] = 1

estimator = neighbors.KNeighborsClassifier(n_neighbors = 10)

cross_val_score(estimator, X, y).mean()

Двадцать:

In [ ]:
from sklearn.cross_validation import cross_val_score

X_a = np.random.randn(100, 20)
X_a[:, 0] += -1
X_b = np.random.randn(100, 20)
X_b[:, 0] += 1

X = np.concatenate((X_a, X_b), axis = 0)

y = np.zeros(200)
y[:100] = 1

estimator = neighbors.KNeighborsClassifier(n_neighbors = 10)

cross_val_score(estimator, X, y).mean()

Две сотни:

In [ ]:
from sklearn.cross_validation import cross_val_score

X_a = np.random.randn(100, 200)
X_a[:, 0] += -1
X_b = np.random.randn(100, 200)
X_b[:, 0] += 1

X = np.concatenate((X_a, X_b), axis = 0)

y = np.zeros(200)
y[:100] = 1

estimator = neighbors.KNeighborsClassifier(n_neighbors = 10)

cross_val_score(estimator, X, y).mean()

Заплатка:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

X_norm = normalize(X)
X_low_dim = PCA(10).fit_transform(X_norm)

estimator = neighbors.KNeighborsClassifier(n_neighbors = 5)

cross_val_score(estimator, X_low_dim, y).mean()

# Послесловие

In [ ]:
from IPython.display import Image
Image("http://scikit-learn.org/dev/_static/ml_map.png", width=800)